In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Using TensorFlow backend.


In [3]:
path_to_file =  tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
#### Load your own data
#from google.colab import files  
#path_to_file = list(files.upload().keys())[0]

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Encoding

In [0]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


### Cleaning Trainig Examples

In [0]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [0]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Model Building

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Creating Loss Function

In [17]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 1.86759233e-03  1.16635207e-03 -1.26199413e-03 ...  8.30300432e-03
   -1.17582572e-03  6.26885146e-03]
  [ 3.43566039e-03  2.15381524e-03 -2.20989296e-03 ...  1.48541145e-02
   -1.26740034e-03  1.10950591e-02]
  [ 1.15922401e-02  2.42568925e-03 -4.69739223e-03 ...  1.20851072e-02
   -3.05597857e-03  1.11515103e-02]
  ...
  [ 1.49235278e-02  1.22489082e-03 -6.39764965e-03 ... -1.18943723e-03
   -5.30797895e-03 -1.10750506e-03]
  [ 9.53012239e-03  4.62089991e-03  2.36015953e-03 ... -3.89344711e-03
   -3.06534325e-03 -1.19954208e-03]
  [ 1.54801747e-02  3.19371326e-03 -1.48654915e-03 ... -1.85930182e-03
   -4.68004402e-03  1.69091951e-03]]

 [[-1.35213998e-03  3.78786679e-03  7.18956813e-03 ... -2.11322587e-03
   -4.08747117e-04 -8.98380764e-04]
  [-4.51066764e-04 -2.08223448e-03  6.33154018e-03 ... -5.69502963e-03
    5.08951780e-05 -3.73539561e-03]
  [ 8.37184209e-03 -1.65615696e-04  1.05197518e-03 ... -2.51463894e-03
   -3.31108691e-03  1.37647381e-04]
  ...
  [ 7.875

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00186759  0.00116635 -0.00126199 ...  0.008303   -0.00117583
   0.00626885]
 [ 0.00343566  0.00215382 -0.00220989 ...  0.01485411 -0.0012674
   0.01109506]
 [ 0.01159224  0.00242569 -0.00469739 ...  0.01208511 -0.00305598
   0.01115151]
 ...
 [ 0.01492353  0.00122489 -0.00639765 ... -0.00118944 -0.00530798
  -0.00110751]
 [ 0.00953012  0.0046209   0.00236016 ... -0.00389345 -0.00306534
  -0.00119954]
 [ 0.01548017  0.00319371 -0.00148655 ... -0.0018593  -0.00468004
   0.00169092]], shape=(100, 65), dtype=float32)


In [20]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 1.8675923e-03  1.1663521e-03 -1.2619941e-03 -7.7304128e-04
 -3.6889396e-03 -3.1224226e-03 -6.2512700e-06 -1.0129604e-03
 -2.3268405e-03 -1.7557553e-03 -4.4967742e-03  4.0311012e-03
  3.6162320e-03  3.3651455e-03  2.8898441e-03 -3.0820146e-03
  5.2067800e-04  6.9070007e-03 -5.9069847e-03 -2.0456696e-03
  2.6308757e-03  1.1827690e-03 -2.1901811e-03  3.5341026e-03
 -2.4618772e-03  2.9818842e-03  3.7999549e-03  2.9797293e-03
 -3.5968430e-03 -1.0368756e-03  2.5818821e-03 -1.0411162e-03
  9.5435902e-03  1.1606510e-03  1.1052074e-03 -1.6543310e-03
 -4.8429356e-03  2.8299645e-03  1.8443390e-03 -2.6749824e-03
  6.1664334e-04 -2.4219442e-03 -2.4602129e-03  3.8845639e-04
 -2.1696670e-03  3.1545360e-03  3.8284953e-03 -2.4794876e-03
  1.3281568e-04 -1.7505337e-03  6.1780261e-03 -3.8986928e-03
  7.1659451e-04  2.6617052e-03 -4.4529377e-03  7.1956581e-03
  1.3710214e-03 -8.4228709e-04 -1.9686781e-03  6.8129972e-04
 -6.4400840e-04  1.7109374e-03  8.3030043e-03 -1.1758257e-03
  6.268851

In [21]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"PuaQh'RWGUPHnyUoplS&Q,F:K CKobHvp&GhmFhNcZ-Hc,xpF:I;nXgx.HX3'I\n\nQby IQfEfs.qF RT'csyTkoYdxAm&TTD :Bc"

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Model Compiling

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Training

In [25]:
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/100
172/172 [==============================] - 13s 76ms/step - loss: 2.7529
Epoch 2/100
172/172 [==============================] - 12s 67ms/step - loss: 2.0721
Epoch 3/100
172/172 [==============================] - 12s 67ms/step - loss: 1.8140
Epoch 4/100
172/172 [==============================] - 12s 67ms/step - loss: 1.6579
Epoch 5/100
172/172 [==============================] - 12s 68ms/step - loss: 1.5582
Epoch 6/100
172/172 [==============================] - 12s 67ms/step - loss: 1.4902
Epoch 7/100
172/172 [==============================] - 12s 67ms/step - loss: 1.4407
Epoch 8/100
172/172 [==============================] - 12s 67ms/step - loss: 1.4032
Epoch 9/100
172/172 [==============================] - 12s 67ms/step - loss: 1.3721
Epoch 10/100
172/172 [==============================] - 12s 67ms/step - loss: 1.3457
Epoch 11/100
172/172 [==============================] - 11s 67ms/step - loss: 1.3220
Epoch 12/100
172/172 [==============================] 

### Load the model

In [0]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [0]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [28]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored

### Gnerating Text

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [30]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: ok I am legend
ok I am legend for her these abundantagable, rain; poor gracious daughter
Reed with our loss.

Second Citizen:
We must thou not garlians, be but sworn my love!
butthe baddest.

KING EDWARD IV:
Ay, but thou wert the king. To shame thee, York
Ure sword, cond us, so done!

GREMIO:
Both holo do grue stile: I'll go along with thee.
Now he did not ask, Pompey; in this our noble lord
The heavens give body's palm therewith Clarence with her cily news,
Swifted to my duty, as my rage,
That being laps the wind-short of the death kindred,
In whom for Edward's ground, I thought
The visitation which we small grow
To see my stood standers and my life tood Katharina, the rock, pale
Of go a-hood! who's there?

BALTHASAR:

ISABELLA:
I thank you for this ill I swear to way,
I shall the queen i' tower;
Unless they bear me 
